In [1]:
import importlib.util
import subprocess
import sys

def check_and_install(package_name):
    if importlib.util.find_spec(package_name) is not None:
        print(f"模塊 '{package_name}' 已安裝")
    else:
        print(f"模塊 '{package_name}' 未安裝，正在安裝...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"模塊 '{package_name}' 安裝完成")

# 檢查並安裝
check_and_install('google-generativeai')
check_and_install('sklearn')


模塊 'google-generativeai' 未安裝，正在安裝...
模塊 'google-generativeai' 安裝完成
模塊 'sklearn' 已安裝


In [2]:
import google.generativeai as genai
import os
from sklearn.metrics import accuracy_score, cohen_kappa_score
from collections import Counter

In [ ]:
API_KEY = ''

# 提示使用者輸入 API 字串
key = input("請輸入您的 API 金鑰：")

# 驗證輸入是否為空
if key.strip():  # 檢查輸入是否非空
    print(f"您輸入的 API 金鑰是：{key}")
    API_KEY = key
else:
    print("未輸入 API 金鑰，請重新執行並輸入")

In [4]:
# 設定 Google Gemini API 金鑰
genai.configure(api_key=API_KEY)
# 初始化 Gemini 模型
model = genai.GenerativeModel('gemini-1.5-flash')

In [5]:
class LLM_Agent:
    def __init__(self, name, personality, strategy, knowledge):
        self.name = name
        self.personality = personality
        self.strategy = strategy
        self.knowledge = knowledge

    def respond(self, message, context):
        prompt = self.create_prompt(message, context)
        response = model.generate_content(prompt)
        sentiment = self.analyze_sentiment(response.text.strip())
        return {
            "agent": self.name,
            "personality": self.personality,
            "response": response.text.strip(),
            "sentiment": sentiment
        }

    def create_prompt(self, message, context):
        prompt_base = f"You are a {self.personality} agent. You have expertise in the following areas: {', '.join(self.knowledge)}.\n"
        prompt_context = f"Here is the previous context of the discussion: '{context}'\n"
        prompt_question = f"Based on your knowledge, provide a response to: '{message}'\n"

        if self.strategy == "logic":
            return prompt_base + prompt_context + "Focus on providing a logical and data-driven solution.\n" + prompt_question
        elif self.strategy == "creative":
            return prompt_base + prompt_context + "Focus on providing an innovative and outside-the-box solution.\n" + prompt_question
        elif self.strategy == "cautious":
            return prompt_base + prompt_context + "Focus on identifying potential risks and concerns.\n" + prompt_question
        elif self.strategy == "optimistic":
            return prompt_base + prompt_context + "Focus on providing an encouraging and solution-oriented response.\n" + prompt_question
        else:
            return prompt_base + prompt_context + prompt_question

    def analyze_sentiment(self, text):
        """
        簡單的情感分析：假設根據回應內容中是否包含正面或負面詞彙來判斷情感
        """
        positive_keywords = ["提升", "優化", "改善", "進步", "鼓勵"]
        negative_keywords = ["風險", "問題", "挑戰", "不足", "缺陷"]

        if any(word in text for word in positive_keywords):
            return "positive"
        elif any(word in text for word in negative_keywords):
            return "negative"
        else:
            return "neutral"

class LLM_MultiAgents:
    def __init__(self):
        self.agents = [
            LLM_Agent("LogicMaster", "理性", "logic", [
                "Differential Privacy", "Secure Multi-Party Computation",
                "Federated Averaging Algorithm", "Distributed System Optimization"
            ]),
            LLM_Agent("CreativeThinker", "創意", "creative", [
                "Generative Adversarial Networks (GAN)", "Semi-Supervised Learning",
                "Stochastic Augmentation Strategies", "Graph-Based Federated Learning"
            ]),
            LLM_Agent("CautiousAnalyst", "謹慎", "cautious", [
                "Risk Analysis in Federated Learning", "Convergence and Stability Factors",
                "Resource Cost Analysis", "Common Attack Vectors"
            ]),
            LLM_Agent("OptimisticPlanner", "樂觀", "optimistic", [
                "Progressive Privacy Enhancement", "Federated Learning Applications",
                "Rapid Deployment and Testing Strategies", "Efficient Collaborative Optimization"
            ]),
        ]
        self.sentiments = []  # 用於存儲所有回應的情感標籤

    def simulate_discussion(self, que, rounds=2):
        context = ""
        for round_num in range(1, rounds + 1):
            print(f"--- 第 {round_num} 輪討論 ---\n")
            for agent in self.agents:
                result = agent.respond(que, context)
                response, sentiment = result["response"], result["sentiment"]
                print(f"{result['agent']} ({result['personality']}): {response}")
                context += f"{response}\n"
                self.sentiments.append(sentiment)
            print("-" * 50)

    def evaluate_results(self):
        """
        評估情感分析的指標，包括準確率、一致性和 Kappa 值
        """
        # 假設真實情感標籤為 'positive'，模擬中可以根據具體場景調整
        true_sentiments = ["positive"] * len(self.sentiments)

        # 計算準確率
        accuracy = accuracy_score(true_sentiments, self.sentiments)

        # 計算情感一致性
        sentiment_counts = Counter(self.sentiments)
        most_common_sentiment = sentiment_counts.most_common(1)[0][0]
        consistency = sentiment_counts[most_common_sentiment] / len(self.sentiments)

        # 計算 Cohen’s Kappa
        kappa = cohen_kappa_score(true_sentiments, self.sentiments)

        print("\n--- 評估結果 ---")
        print(f"情感準確率 (Accuracy): {accuracy:.2f}")
        print(f"情感一致性 (Consistency): {consistency:.2f}")
        print(f"Cohen's Kappa: {kappa:.2f}")

In [6]:
# 測試 Multi-Agent 系統
question1 = "如何提升聯邦學習的隱私保護和模型效能？"
multi_agents = LLM_MultiAgents()
multi_agents.simulate_discussion(question1, rounds=3)
multi_agents.evaluate_results()

--- 第 1 輪討論 ---

LogicMaster (理性): 提升聯邦學習的隱私保護和模型效能需要在隱私增强技术和模型优化策略之间取得平衡。单纯追求高隐私保护可能会牺牲模型精度，反之亦然。因此，一个合理的策略需要综合考虑以下几个方面：

**一、增强隐私保护：**

1. **更强的差分隐私机制:**  联邦学习天然适合差分隐私的应用。  可以考虑采用更高级的差分隐私机制，例如：
    * **自适应噪声机制:** 根据数据的敏感度动态调整噪声级别，在保证隐私的同时最大限度地保留有用信息。
    * **局部差分隐私:**  在客户端本地添加噪声，而不是在聚合结果上添加噪声，进一步增强隐私保护。  这需要更复杂的客户端计算和更有效的噪声机制设计来避免过大的精度损失。
    * **组合差分隐私:**  将多个差分隐私机制组合使用，例如将局部差分隐私与全局差分隐私结合，以获得更强的隐私保护能力。
    * **探索后量子密码学:**  考虑使用抗量子计算攻击的加密技术来增强隐私保护。

2. **安全多方计算 (MPC):**  利用MPC技术，在不泄露原始数据的情况下进行模型训练。  这比差分隐私更强，但也可能带来更高的计算开销。  可以考虑使用高效的MPC协议，例如基于秘密分享的协议或同态加密协议。

3. **同态加密:**  使用同态加密技术对数据进行加密，然后在加密数据上进行模型训练。  训练完成后，再解密得到模型参数。  这需要选择合适的同态加密方案，并优化加密和解密的效率。

4. **联邦学习架构设计:**  谨慎设计联邦学习的架构，例如：
    * **选择合适的聚合方式:**  考虑使用更安全的聚合算法，例如基于安全多方计算的聚合算法。
    * **限制参与者的数量:**  减少参与者的数量可以降低数据泄露的风险。
    * **选择合适的通信协议:**  选择安全的通信协议，例如使用TLS加密通信。


**二、提升模型效能：**

1. **优化模型架构:**  选择合适的模型架构，例如更轻量级的模型或者针对特定任务优化的模型，以提高模型的效率和精度。

2. **改进联邦平均算法:**  采用更先进的联邦平均算法，例如：
    * **FedProx:**  解决客户端数据非独立同分布 (Non-IID) 的问题。


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:758: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
